# Generate Only Parameter Cards
For batch running on noether. Also see alex_jobsub.sh or whatever I rename it to.

In [6]:
import os
import numpy as np

### Code

In [7]:
def generate_paramater_card(ma, dm_mass, neutral_meson):
    """Generate a parameter card for this ma and dt_ratio

    Args:
        ma (str): mass of the dark photon
        dm_mass (str): mass of the dark matter particle (chi)
        neutral_meson (str): the neutral meson that produces the chi particles
    Returns:
        param_card_name (str): the name of the parameter card file
    """

    new_param_card = ""
    with open("../Noether/example.dat", "r") as f:
        for line in f:
            if "#" in line:
                # skip comments but include them in the new file
                # (there may be the word "run" in comments?)
                new_param_card += line
            elif "samplesize" in line[0:15]: 
                new_line = "samplesize " + str(NUM_DARK_SCALARS) + "\n"
                new_param_card += new_line
            elif "dark_matter_mass" in line[0:16]:
                new_line = "dark_matter_mass " + str(dm_mass) + "\n"
                new_param_card += new_line
            elif "dark_photon_mass" in line[0:20]:
                new_line = "dark_photon_mass " + str(ma) + "\n"
                new_param_card += new_line
            elif "root_file" == line[0:9]:
                new_line = "root_file " + f"/gluster/data/microboone/electron_scatter/{CHI_TYPE}/{neutral_meson}_{CHI_TYPE}_ma_{ma}_dt_{DT_RATIO_STR}.root\n"
                new_param_card += new_line
            elif "decay_type" in line[0:12]:
                new_line = "decay_type " + CHI_TYPE + "\n"
                new_param_card += new_line
            elif "production_channel" in line:
                new_line = "production_channel " + neutral_meson + "_decay\n"
                new_param_card += new_line
            elif "particle_list_file" in line:
                new_line = f"particle_list_file ../../../neutral_meson_flux/{neutral_meson}s.dat\n"
                new_param_card += new_line
            elif "summary_file" in line[0:14]:
                new_line = "summary_file bdnmc_summary.dat\n"
                new_param_card += new_line
            elif ("particle_list_weight" in line) and (neutral_meson=="eta"):
                new_line = "particle_list_weight true\nmeson_per_pi0 0.09781263276\n\n"
                new_param_card += new_line
            elif ("particle_list_weight" in line) and (neutral_meson=="pi0"):
                new_line = "particle_list_weight true\nmeson_per_pi0 1\n\n"
                new_param_card += new_line
            else:
                new_param_card += line
    
    # write the new_parameter_card to a .dat file
    param_card_id = f"../Noether/{CHI_TYPE}/{neutral_meson}_{CHI_TYPE}_ma_{ma}_dt_{DT_RATIO_STR}.dat"
    with open(param_card_id, "w") as f:
        f.write(new_param_card)
    
    param_card_name = f"{CHI_TYPE}/{neutral_meson}_{CHI_TYPE}_ma_{ma}_dt_{DT_RATIO_STR}.dat"
    return param_card_name

### Math Check

$$ dt = M_\chi / M_{A'} $$
$$ M_\eta > 2 M_\chi $$
$$ \therefore M_\eta > 2 \times dt \times M_{A'}$$
$$ \therefore \text{max}(M_{A'}) < \frac{M_\eta}{2 \times dt} $$

(And similar for $M_\pi$)

In [8]:
def gen_mass_points(dt_ratio, neutral_meson):
    """
     * We want to generate files at sensible mass points.
     * But limits are effected by dt ratio
     * This is limited too the mass of an eta > 2 * m_chi
     * dt_ratio = m_chi / m_A'
     * so m_A'_max = m_chi / dt_ratio

    Returns:
        list : generated mass points
    """
    m_eta_meson = 0.547 # GeV (actually 0.547862, round slightly down to be safe)
    m_pi0_meson = 0.1349
    min_ma = 0.01 # GeV so 10 MeV
    
    if neutral_meson == "pi0":
        ma_max = m_pi0_meson / (2.0 * dt_ratio)

    elif neutral_meson == "eta":
        ma_max = m_eta_meson / (2.0 * dt_ratio)
        
    else:
        print("err: invalid neutral meson string")
        return -999.9


    # generate mass points
    # we want 10 in the range 0.01 to 0.1
    # and 1 every 0.05 from 0.1 to ma_max
    if ma_max < 0.1:
        mass_points = np.arange(min_ma, ma_max, 0.01).tolist()
    else:
        mass_points = np.arange(min_ma, 0.1, 0.01).tolist()
        mass_points += np.arange(0.1, ma_max, 0.1).tolist()

    # round each mass point to 2 decimal places
    
    mass_points = [round(ma, 2) for ma in mass_points]
    
    #order numbers in ascending order
    mass_points.sort()
    
    return mass_points

$$ dt = \frac{M_\chi}{M_{A'}}$$
$$ M_\chi = dt \times M_{A'} $$

In [9]:
def main():
    # Need a .txt file with list of parameter cards for noether

        netural_mesons = ["pi0", "eta"]
        for meson in netural_mesons:
            with open(f"../Noether/card_list_{CHI_TYPE}_{meson}_{DT_RATIO_STR}.txt", "w") as f:
                mass_points = gen_mass_points(DT_RATIO, meson)
                for ma in mass_points:
                    dm_mass = DT_RATIO * ma
                    param_card_id = generate_paramater_card(ma, dm_mass, meson)

                    f.write(param_card_id + "\n")



In [10]:
PATH = os.getcwd() + "/../../DarkTridentGen/"
NUM_DARK_SCALARS = 10_000
CHI_TYPE_LIST = ["scalar", "fermion"]
DT_RATIO_LIST = [0.6, 1./3., 2.0]
for c in CHI_TYPE_LIST:
    for dtr in DT_RATIO_LIST:
        DT_RATIO = dtr
        CHI_TYPE = c
        DT_RATIO_STR = str(round(DT_RATIO, 2))
        print(DT_RATIO_STR)
        main()

0.6
0.33
2.0
0.6
0.33
2.0
